In [1]:
#os.environ['OPENAI_API_KEY'] = <your-api-key>
import openai
import os

api_key = os.getenv('OPENAI_API_KEY')

https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_5_to_9.ipynb

In [2]:

import pandas as pd
import re
def data_loader():
  """Loads data from CSV files, performs data cleaning and feature engineering.

  Returns:
    pandas.DataFrame: The processed hotel reviews dataset.
  """

  # Read hotel reviews data
  Hotel_Reviews = pd.read_csv("../Data/Hotel_Reviews.csv")

  # Read country data
  countries = pd.read_csv("../Data/countries.csv")

  # Add a "Country" column with NA values
  Hotel_Reviews["Country"] = pd.NA

  # Assign country based on pattern matching in Hotel_Address
  for country_name in countries["name"]:
    pattern = re.compile(country_name, re.IGNORECASE)  # Case-insensitive matching
    Hotel_Reviews.loc[Hotel_Reviews["Hotel_Address"].str.contains(pattern), "Country"] = country_name

 
  # Convert Review_Date to datetime and extract features
  Hotel_Reviews["date_object"] = pd.to_datetime(Hotel_Reviews["Review_Date"], format="%m/%d/%Y")
  #Hotel_Reviews["time"] = Hotel_Reviews["date_object"].astype(int) / 10**9  # Convert to Unix timestamp
  Hotel_Reviews["month"] = Hotel_Reviews["date_object"].dt.month
  Hotel_Reviews["num_date_object"] = Hotel_Reviews["date_object"].dt.day_of_year / 365  # Normalize by days in a year

  return Hotel_Reviews

# Load the processed data
Hotel_Reviews = data_loader()

C:\Users\alan_\AppData\Local\Temp\ipykernel_15972\278692378.py:22: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  Hotel_Reviews.loc[Hotel_Reviews["Hotel_Address"].str.contains(pattern), "Country"] = country_name
C:\Users\alan_\AppData\Local\Temp\ipykernel_15972\278692378.py:22: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  Hotel_Reviews.loc[Hotel_Reviews["Hotel_Address"].str.contains(pattern), "Country"] = country_name
C:\Users\alan_\AppData\Local\Temp\ipykernel_15972\278692378.py:22: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  Hotel_Reviews.loc[Hotel_Reviews["Hotel_Address"].str.contains(pattern), "Country"] = country_name
C:\Users\alan_\AppData\Local\Temp\ipykernel_15972\278692378.py:22: UserWarning: This pattern is inte

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,...,Reviewer_Score,Tags,days_since_review,lat,lng,Country,date_object,month,num_date_object,MergedColumn_topic
405,1 15 Templeton Place Earl s Court Kensington a...,244,8/3/2017,8.5,K K Hotel George,Ireland,Really nothing negative to say about this hotel,9,1831,Very comfortable beds smart bathroom good sho...,...,9.6,"[' Leisure trip ', ' Group ', ' Classic Twin R...",0 days,51.491888,-0.194971,United Kingdom,2017-08-03,8,0.589041,Very comfortable beds smart bathroom good sho...
406,1 15 Templeton Place Earl s Court Kensington a...,244,8/2/2017,8.5,K K Hotel George,United States of America,No coffee or tea in the room or lobby No one ...,52,1831,No Positive,...,7.1,"[' Leisure trip ', ' Couple ', ' Classic Twin ...",1 days,51.491888,-0.194971,United Kingdom,2017-08-02,8,0.586301,No Positive. No coffee or tea in the room or ...
407,1 15 Templeton Place Earl s Court Kensington a...,244,7/20/2017,8.5,K K Hotel George,United States of America,Room was pretty small for the three of us,11,1831,The staff was very friendly and helpful in or...,...,9.2,"[' Leisure trip ', ' Family with young childre...",14 days,51.491888,-0.194971,United Kingdom,2017-07-20,7,0.550685,The staff was very friendly and helpful in or...
408,1 15 Templeton Place Earl s Court Kensington a...,244,7/19/2017,8.5,K K Hotel George,Australia,Nothing really,4,1831,We stayed at the hotel at a friend s recommen...,...,9.6,"[' Leisure trip ', ' Couple ', ' Classic Doubl...",15 days,51.491888,-0.194971,United Kingdom,2017-07-19,7,0.547945,We stayed at the hotel at a friend s recommen...
409,1 15 Templeton Place Earl s Court Kensington a...,244,7/18/2017,8.5,K K Hotel George,United States of America,Only cab service to airport is expensive comp...,16,1831,Super location on Division underground line d...,...,8.3,"[' Leisure trip ', ' Group ', ' Classic Twin R...",16 days,51.491888,-0.194971,United Kingdom,2017-07-18,7,0.545205,Super location on Division underground line d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515408,Wrights Lane Kensington and Chelsea London W8 ...,1172,8/5/2015,7.8,Holiday Inn London Kensington,Belgium,Internet should never ever be payable in a ho...,11,5945,No Positive,...,9.6,"[' Leisure trip ', ' Couple ', ' Standard Doub...",729 day,51.499981,-0.192879,United Kingdom,2015-08-05,8,0.594521,No Positive. Internet should never ever be pa...
515409,Wrights Lane Kensington and Chelsea London W8 ...,1172,8/4/2015,7.8,Holiday Inn London Kensington,United Kingdom,Rooms a little small but adequate,7,5945,Loved the location friendly helpful staff,...,10.0,"[' Leisure trip ', ' Couple ', ' Superior Doub...",730 day,51.499981,-0.192879,United Kingdom,2015-08-04,8,0.591781,Loved the location friendly helpful staff. R...
515410,Wrights Lane Kensington and Chelsea London W8 ...,1172,8/4/2015,7.8,Holiday Inn London Kensington,United Kingdom,The room wasn t what was described on website...,24,5945,Location was good,...,5.0,"[' Leisure trip ', ' Couple ', ' Executive Dou...",730 day,51.499981,-0.192879,United Kingdom,2015-08-04,8,0.591781,Location was good . The room wasn t what was...
515411,Wrights Lane Kensington and Chelsea London W8 ...,1172,8/4/2015,7.8,Holiday Inn London Kensington,United Kingdom,Bed room service was very late,7,5945,No Positive,...,8.8,"[' Couple ', ' Standard Double Room ', ' Staye...",730 day,51.499981,-0.192879,United Kingdom,2015-08-04,8,0.591781,No Positive. Bed room service was very late


In [29]:
from langchain_text_splitters import RecursiveCharacterTextSplitter



# Merge the columns using string concatenation
Hotel_Reviews['MergedColumn_topic'] = (
    Hotel_Reviews['Positive_Review'] + 
    '. ' + Hotel_Reviews['Negative_Review'] 
)
# Select the first 100 rows of the merged column
Hotel_Reviews_UK =Hotel_Reviews[Hotel_Reviews["Country"]== "United Kingdom"]
used_data = Hotel_Reviews_UK['Positive_Review']#Hotel_Reviews['MergedColumn_topic']#[:2000]

# Specify the file name
file_name = "used_data.txt"

# Save the data to a text file
with open(file_name, 'w') as f:
    for line in used_data:
        f.write(line + '\n')

    


#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [1]:
!pip install bertopic


  Using cached plotly-5.24.1-py3-none-any.whl.metadata (7.3 kB)
Using cached plotly-5.24.1-py3-none-any.whl (19.1 MB)
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 14.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/28.1 MB ? eta -:--:--
   - -------------------------------------- 0.8/28.1 MB 3.4 MB/s eta 0:00:09
   --- ------------------------------------ 2.4/28.1 MB 5.4 MB/s eta 0:00:05
   ----- ---------------------------------- 3.9/28.1 MB 6.2 MB/s eta 0:00:04
   ------- -------------------------------- 5.2/28.1 MB 6.1 MB/s eta 0:00:04
   ---------- ----------------------------- 7.1/28.1 MB 6.7 MB/s eta 0:00:04
   ------------- -------------------------- 9.2/28.1 MB 7.3 MB/s eta 0:00:03
   ------------------ --------------------- 12.8/28.1 MB 8.8 MB/s eta 0:00:02
   ------------------------ --------------- 17.3/28.1 MB 10.3 MB/s eta 0:00:02
   ------------------------------ --------- 21.

In [30]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
 
#docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']
docs = used_data.tolist()
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs)

In [32]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,88366,-1_coffee_bar_london_we,"[coffee, bar, london, we, shower, with, bathro...",[ The whole experience was great Although an u...
1,0,19639,0_positive_no_commment_bracket,"[positive, no, commment, bracket, tentative, p...","[No Positive, No Positive, No Positive]"
2,1,6779,1_location_amaing_locationthe_golf,"[location, amaing, locationthe, golf, rocks, g...","[ Location, Location, location]"
3,2,3551,2_were_room_helpful_polite,"[were, room, helpful, polite, staff, rooms, up...",[ Staff very friendly and helpful and the room...
4,3,3035,3_birthday_cake_anniversary_wedding,"[birthday, cake, anniversary, wedding, card, b...",[ Staff were lovely left me a birthday cake ca...
...,...,...,...,...,...
2265,2264,10,2264_convenient_simple_book_reasonable,"[convenient, simple, book, reasonable, handy, ...","[ Convenient and clean, Clean and convenient,..."
2266,2265,10,2265_lily_langtry_sumptuous_items,"[lily, langtry, sumptuous, items, complementar...",[ Lovely room with very nice complementary ite...
2267,2266,10,2266_midway_hyde_hydepark_destination,"[midway, hyde, hydepark, destination, undergro...",[ great destination very close to hydepark and...
2268,2267,10,2267_matinee_joanna_10pm_basement,"[matinee, joanna, 10pm, basement, performance,...",[ Reception staff not very friendly at check i...


In [43]:
topic_model.get_topic(6
)

[('everything', 0.1257731944293152),
 ('almosut', 0.0041883140181031755),
 ('goring', 0.0030940197028826023),
 ('xxxx', 0.0030191222469659),
 ('honestly', 0.0029992733431656684),
 ('surprising', 0.0021511495931072896),
 ('lol', 0.0021511495931072896),
 ('noticed', 0.0018329923283696287),
 ('smell', 0.0012069951216006898),
 ('change', 0.0008034613635838656)]

In [24]:
topic_model.get_document_info(docs)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,Very comfortable beds smart bathroom good sho...,-1,-1_the_was_and_in,"[the, was, and, in, of, we, but, were, room, h...",[ This hotel had excellent staff the front des...,the - was - and - in - of - we - but - were - ...,0.000000,False
1,No Positive. No coffee or tea in the room or ...,78,78_milk_coffee_tea_positive,"[milk, coffee, tea, positive, making, machine,...",[No Positive. There was not enough coffee and...,milk - coffee - tea - positive - making - mach...,0.303844,False
2,The staff was very friendly and helpful in or...,-1,-1_the_was_and_in,"[the, was, and, in, of, we, but, were, room, h...",[ This hotel had excellent staff the front des...,the - was - and - in - of - we - but - were - ...,0.000000,False
3,We stayed at the hotel at a friend s recommen...,-1,-1_the_was_and_in,"[the, was, and, in, of, we, but, were, room, h...",[ This hotel had excellent staff the front des...,the - was - and - in - of - we - but - were - ...,0.000000,False
4,Super location on Division underground line d...,-1,-1_the_was_and_in,"[the, was, and, in, of, we, but, were, room, h...",[ This hotel had excellent staff the front des...,the - was - and - in - of - we - but - were - ...,0.000000,False
...,...,...,...,...,...,...,...,...
262296,No Positive. Internet should never ever be pa...,328,328_wifi_internet_positive_whifi,"[wifi, internet, positive, whifi, pay, fi, wi,...",[No Positive. I stayed in a lot of mom and po...,wifi - internet - positive - whifi - pay - fi ...,0.493281,False
262297,Loved the location friendly helpful staff. R...,92,92_small_rooms_friendly_location,"[small, rooms, friendly, location, staff, size...",[ Location great staff helpful and friendly. ...,small - rooms - friendly - location - staff - ...,0.576340,False
262298,Location was good . The room wasn t what was...,-1,-1_the_was_and_in,"[the, was, and, in, of, we, but, were, room, h...",[ This hotel had excellent staff the front des...,the - was - and - in - of - we - but - were - ...,0.000000,False
262299,No Positive. Bed room service was very late,-1,-1_the_was_and_in,"[the, was, and, in, of, we, but, were, room, h...",[ This hotel had excellent staff the front des...,the - was - and - in - of - we - but - were - ...,0.000000,False


In [54]:
from bertopic.representation import KeyBERTInspired

# Fine-tune your topic representations
representation_model = KeyBERTInspired()
topic_model = BERTopic(representation_model=representation_model)

In [35]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

openai.api_key = api_key #for simple openai
OPENAI_API_KEY = api_key #for langchain
!export OPENAI_API_KEY=OPENAI_API_KEY

Der Befehl "export" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [ ]:
import openai
from bertopic.representation import OpenAI

# Fine-tune topic representations with GPT
client = openai.OpenAI(api_key=OPENAI_API_KEY)
#representation_model = OpenAI(client, model="gpt-4o-mini", chat=True)
#topic_model = BERTopic(representation_model=representation_model)

In [26]:
topic_model.fit(docs)

In [33]:
topic_model.visualize_barchart()



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.00010393679757235385, 0.00012834401730507083,
                    0.0001892294494904965, 0.03361281722096753,
                    0.04082320843543864],
              'xaxis': 'x',
              'y': [tentative  , bracket  , commment  , no  , positive  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.0008310682905407387, 0.0008578211913440493,
                    0.001094399732391158, 0.001094399732391158,
                    0.01793402645867605],
              'xaxis': 'x2',
              'y': [rocks  , golf  , locationthe  , amaing  , location  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.0009767554160788464, 0.0010050927316695452,
                    0.0010248736565171878, 0.00111488320162345,
                    0.0014587678183509288],
              'xaxis': 'x3',
              'y': [staff  , polite  , helpful  , room  , were  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.004273900831681877, 0.004509889013423062,
                    0.0059780874210394, 0.006926581580341949, 0.008857057071908423],
              'xaxis': 'x4',
              'y': [card  , wedding  , anniversary  , cake  , birthday  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.0010326925523481928, 0.0010809034828848467,
                    0.001240609310932745, 0.001394331833983763,
                    0.0019153708240465786],
              'xaxis': 'x5',
              'y': [breakfasts  , delicious  , choice  , hotel  , breakfast  ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.0030569424454051754, 0.003086568703389768,
                    0.004961124322883412, 0.0054092199023352844,
                    0.01956430712095058],
              'xaxis': 'x6',
              'y': [lancaster  , station  , express  , heathrow  , paddington  ],
              'yaxis': 'y6'},
             {'marker': {'color': '#F0E442'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.0029992733431656684, 0.0030191222469659,
                    0.0030940197028826023, 0.0041883140181031755,
                    0.1257731944293152],
              'xaxis': 'x7',
              'y': [honestly  , xxxx  , goring  , almosut  , everything  ],
              'yaxis': 'y7'},
             {'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.001464522932742324, 0.0018292443092111446,
                    0.0019520187724321651, 0.002083299716678032,
                    0.002917587878944903],
              'xaxis': 'x8',
              'y': [this  , best  , hotels  , favourite  , london  ],
              'yaxis': 'y8'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 0',
                                'x': 0.0875,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 1',
                                'x': 0.36250000000000004,
                          

In [28]:
from langchain_ollama.llms import OllamaLLM

# Initialize the LLM
llm=OllamaLLM(model="llama3.2", max_tokens=4000)

# Group reviews by hotel
grouped_reviews = Hotel_Reviews.groupby('Hotel_Name')['MergedColumn_topic'].apply(lambda x: ' '.join(x))

# Summarize reviews for each hotel
summaries = {}
for hotel, reviews in grouped_reviews.items():
    summary = llm.summarize(reviews)
    summaries[hotel] = summary

# Convert summaries to a DataFrame or save them as needed
summaries_df = pd.DataFrame(list(summaries.items()), columns=['Hotel_Name', 'Summary'])

AttributeError: 'OllamaLLM' object has no attribute 'summarize'